# Rossmann Store Sales Forecasting
**Business Analytics (Spring, 2017)**  
서한빈(석사2) @ Data Science, Seoultech  

Submit Date: 2017.3.22. (Assignment 03)

* * *  
## 0.1. Problem
### 유럽의 올리브영이라는 드럭스토어 Rossmann 매출 예측 문제
지난 과제에서 데이터를 탐색하고, 매출을 예측하는 문제를 해결해 보았다.  
결과는 좋지 않았지만, 주어진 변수를 어떻게 활용할 수 있을지 고민해 볼 수 있는 시간이였다.  
이번 과제에서는 `Promo2` 관련 변수를 활용해보고, 마지막으로 모델의 성능을 향상 시켜야 한다.  

### Assignment's Problem
1. Add Variables related with `Promo2`  
  Use 'Promo2', 'Promo2Since[Year/Week]', 'PromoInterval'
2. Use Store  
  How to differentiate different store ?
3. Any other ideas
4. **Build yout own models**  
  Compare before and after
5. Summarize answers as slide  
  Upload your code; Explain your process and summarize result using PowerPoint and upload the slide.

* * *  
## 0.2. import data, preprocessing

In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np

f_dir = "D:/course_2017_spring/BA/Data/W1_Rossmann/"
raw_train = pd.read_csv(f_dir+"train.csv")
raw_store = pd.read_csv(f_dir+"store.csv")
raw_train.loc[raw_train.StateHoliday == 0, 'StateHoliday'] = '0'

df_train = raw_train[raw_train['Open']==1]
df_train = df_train.drop(['Open', 'Customers'], axis=1)

dm_DayOfWeek = pd.get_dummies(df_train['DayOfWeek'], prefix='DayOfWeek')
dm_StateHoliday = pd.get_dummies(df_train['StateHoliday'], prefix='StateHoliday')
df_train[list(set(dm_DayOfWeek))] = dm_DayOfWeek
df_train[list(set(dm_StateHoliday))] = dm_StateHoliday
df_train = df_train.drop(['DayOfWeek', 'StateHoliday'], axis=1)

dm_StoreType = pd.get_dummies(raw_store['StoreType'], prefix='StoreType')
dm_Assortment = pd.get_dummies(raw_store['Assortment'], prefix='Assortment')

df_store = df()
df_store['Store'] = raw_store['Store']
df_store[list(set(dm_StoreType))] = dm_StoreType
df_store[list(set(dm_Assortment))] = dm_Assortment

merge_table = pd.merge(df_train, df_store, on='Store')
merge_table.head()

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,Date,Sales,Promo,SchoolHoliday,DayOfWeek_4,DayOfWeek_1,DayOfWeek_6,DayOfWeek_7,DayOfWeek_2,...,StateHoliday_b,StateHoliday_0,StateHoliday_c,StoreType_c,StoreType_a,StoreType_b,StoreType_d,Assortment_b,Assortment_c,Assortment_a
0,1,2015-07-31,5263,1,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,2015-07-30,5020,1,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,2015-07-29,4782,1,1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1,2015-07-28,5011,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1,2015-07-27,6102,1,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


날짜 형식의 칼럼을 수정.

In [2]:
merge_table['dt_date'] = pd.to_datetime(merge_table['Date'])
merge_table['dt_date'][0]
#del merge_table['Date']

Timestamp('2015-07-31 00:00:00')

`CompetitionOpenSince[Month/Year]` 칼럼 조작:  
> 경쟁업체의 오픈 여부를 단순히 변수화하는 것보다는 관측데이터의 timestampe와의 차이(SinceCompetitionOpen)를 iterval(간격) 변수로 보는 것이 좋겠다.  

업계용어로 '오픈빨'이라고 불리는 것이 영향을 끼칠 것이라고 가정하기 떄문이다.

In [3]:
test_record = merge_table[['Store', 'dt_date', 'Sales']]
test_store = raw_store[['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']]
#test_store.head()
arr_year = test_store['CompetitionOpenSinceYear'].fillna(test_store['CompetitionOpenSinceYear'].max()+1).astype('int').astype('str')
arr_month = test_store['CompetitionOpenSinceMonth'].fillna(test_store['CompetitionOpenSinceMonth'].min()).astype('int').astype('str')
arr_dt = arr_year+'-'+arr_month
test_store['dt_CompetitionOpen'] = pd.to_datetime(arr_dt)
test_store = test_store.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'], axis=1)
#test_store.head()

test_merge = pd.merge(test_record, test_store, on='Store')
test_merge['SinceCompetitionOpen'] = test_merge['dt_date'] - test_merge['dt_CompetitionOpen']
print(test_merge['SinceCompetitionOpen'].min(), "\n", test_merge['SinceCompetitionOpen'].max())
test_merge['SinceCompetitionOpen'] = test_merge['SinceCompetitionOpen'].astype('timedelta64[D]')
test_merge.head()

-1095 days +00:00:00 
 42214 days 00:00:00


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Store,dt_date,Sales,CompetitionDistance,dt_CompetitionOpen,SinceCompetitionOpen
0,1,2015-07-31,5263,1270.0,2008-09-01,2524.0
1,1,2015-07-30,5020,1270.0,2008-09-01,2523.0
2,1,2015-07-29,4782,1270.0,2008-09-01,2522.0
3,1,2015-07-28,5011,1270.0,2008-09-01,2521.0
4,1,2015-07-27,6102,1270.0,2008-09-01,2520.0


In [4]:
import seaborn as sns
sns.distplot(test_merge[test_merge['SinceCompetitionOpen']>0]['SinceCompetitionOpen'])

C:\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


In [5]:
import seaborn as sns
sns.distplot(test_merge[test_merge['SinceCompetitionOpen']<0]['SinceCompetitionOpen'])
#sns.distplot(test_merge[test_merge['SinceCompetitionOpen']>0]['SinceCompetitionOpen'])

C:\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


먼저 `SinceCompetitionOpen` 변수가 *음수* 일때는 경쟁업체가 그 시점에 존재하지 않는다는 것이다.  
이럴경우에는 경쟁업체에 대한 고려는 배제해도 무관 할 것이라고 생각된다.

따라서, `SinceCompetitionOpen` 변수는 *양수* 일때는 과거시점, *음수* 일때는 미래시점.  
0 일때는 현재시점과 동일하기 떄문에, **값이 크면 클수록 영향력은 작아질 것이다** 라고 가정 할 수 있다.  
따라서 *음수* 일때의  `SinceCompetitionOpen` 변수는 모두 max() 값으로 대체할 수 있을 것이다.

In [6]:
#test_merge[test_merge['SinceCompetitionOpen']<0].index
test_merge['SinceCompetitionOpen'][test_merge[test_merge['SinceCompetitionOpen']<0].index] = test_merge['SinceCompetitionOpen'].max()+1

sns.distplot(test_merge[test_merge['SinceCompetitionOpen']>0]['SinceCompetitionOpen'])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


이와 비슷하게 :  
  
`SinceCompetitionOpen` 변수와 동일하게 거리에 대한 변수 `CompetitionDistance` 또한  
멀면 멀수록 영향력이 작다고 생각하고, null 값을 최대값보다 크게 대체할 수 있다.  

In [7]:
#test_merge['CompetitionDistance'].head()
arr_dist = test_merge['CompetitionDistance'].fillna(test_merge['CompetitionDistance'].max()+1)
test_merge['CompetitionDistance'] = arr_dist
test_merge.head()

,Store,dt_date,Sales,CompetitionDistance,dt_CompetitionOpen,SinceCompetitionOpen
0,1,2015-07-31,5263,1270.0,2008-09-01,2524.0
1,1,2015-07-30,5020,1270.0,2008-09-01,2523.0
2,1,2015-07-29,4782,1270.0,2008-09-01,2522.0
3,1,2015-07-28,5011,1270.0,2008-09-01,2521.0
4,1,2015-07-27,6102,1270.0,2008-09-01,2520.0


결과적으로 **`test_merge`** 테이블에서 `CompetitionDistance`와 `SinceCompetitionOpen` 변수를 사용할 수 있을 것으로 보인다.  
메인 데이터셋인 **`merge_table`**에 값을 붙이도록 한다.

In [8]:
merge_table['CompetitionDistance'] = test_merge['CompetitionDistance']
merge_table['SinceCompetitionOpen'] = test_merge['SinceCompetitionOpen']
merge_table.head()

,Store,Date,Sales,Promo,SchoolHoliday,DayOfWeek_4,DayOfWeek_1,DayOfWeek_6,DayOfWeek_7,DayOfWeek_2,...,StoreType_c,StoreType_a,StoreType_b,StoreType_d,Assortment_b,Assortment_c,Assortment_a,dt_date,CompetitionDistance,SinceCompetitionOpen
0,1,2015-07-31,5263,1,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-31,1270.0,2524.0
1,1,2015-07-30,5020,1,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-30,1270.0,2523.0
2,1,2015-07-29,4782,1,1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-29,1270.0,2522.0
3,1,2015-07-28,5011,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-28,1270.0,2521.0
4,1,2015-07-27,6102,1,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-27,1270.0,2520.0


* * *
## 0.3 pilot modeling
모델성능 비교를 위해 1차적으로 `mlr`을 시행하려고 한다.  
먼저 input 데이터와 target 데이터를 분리하고, train 데이터와 test 데이터를 분할하고자 한다.  
- train 데이터는 2013년부터 2014년 까지의 데이터.
- test 데이터는 2015년 상반기(1/1~7/31) 데이터를 사용하고자 한다.

In [9]:
from datetime import datetime as dt
X = merge_table.drop(["Store", "Date", "Sales"], axis=1)
X_train, X_test = X[X['dt_date']<dt(2015,1,1)], X[X['dt_date']>=dt(2015,1,1)]
X_train, X_test = X_train.drop(["dt_date"], axis=1), X_test.drop(["dt_date"], axis=1)
X_train.head()

,Promo,SchoolHoliday,DayOfWeek_4,DayOfWeek_1,DayOfWeek_6,DayOfWeek_7,DayOfWeek_2,DayOfWeek_5,DayOfWeek_3,StateHoliday_a,...,StateHoliday_c,StoreType_c,StoreType_a,StoreType_b,StoreType_d,Assortment_b,Assortment_c,Assortment_a,CompetitionDistance,SinceCompetitionOpen
175,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1270.0,2312.0
176,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1270.0,2311.0
177,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1270.0,2310.0
178,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1270.0,2308.0
179,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1270.0,2305.0


In [10]:
y = merge_table[["Sales", "dt_date"]]
y_train, y_test = y[y['dt_date']<dt(2015,1,1)], y[y['dt_date']>=dt(2015,1,1)]
y_train, y_test = y_train.drop(["dt_date"], axis=1), y_test.drop(["dt_date"], axis=1)
y_train.head()

,Sales
175,2605
176,6466
177,6463
178,6057
179,3659


In [11]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [12]:
y_pred = mlr.predict(X_test)
residual = y_test - y_pred

# The coefficients
print('Coefficients: \n', mlr.coef_)

from sklearn import metrics
R2 = metrics.r2_score(y_test, y_pred)
print('R2 Score: \t', R2)

Coefficients: 
 [[  2.30049989e+03   1.64468851e+02   1.00124533e+03  -1.06257263e+02
   -4.75005130e+02  -4.51703138e+02  -6.27426254e+01  -6.00715862e+01
    1.54534417e+02  -2.26495982e+02  -1.12258802e+02  -3.61994635e+02
    7.00749419e+02  -1.22086197e+03   4.12105630e+03  -1.33670550e+03
   -1.56348883e+03   9.19562158e+02  -2.62662551e+03   1.70706335e+03
   -1.57615607e-02   6.73618751e-04]]
R2 Score: 	 0.213337790076


* * *
## 1. Add Variables related with `Promo2`  
  Use 'Promo2', 'Promo2Since[Year/Week]', 'PromoInterval'

In [13]:
test_record = merge_table[['Store', 'dt_date', 'Sales']]
test_store = raw_store[['Store', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']]
test_store.head()

,Store,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,0,NaN,NaN,NaN
1,2,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,0,NaN,NaN,NaN
4,5,0,NaN,NaN,NaN


In [14]:
for key in ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'] :
    print(test_store[test_store['Promo2']==0][key].value_counts())

Series([], Name: Promo2SinceWeek, dtype: int64)
Series([], Name: Promo2SinceYear, dtype: int64)
Series([], Name: PromoInterval, dtype: int64)


먼저 이진변수인 `Promo2`==0 일때는 나머지 정보가 모두 NaN이므로, 의미가 없다.  
True 일때의 매장에 대해서만 처리가 필요하다.

In [15]:
test_store = test_store[test_store['Promo2']==1]

In [16]:
print("Week:\t", test_store['Promo2SinceWeek'].min(), "\t~\t",test_store['Promo2SinceWeek'].max())
print("Year:\t", test_store['Promo2SinceYear'].min(), "\t~\t",test_store['Promo2SinceYear'].max())

Week:	 1.0 	~	 50.0
Year:	 2009.0 	~	 2015.0


`Promo2SinceWeek` 변수의 의미를 생각해보면, 각각의 매장에서 'Promo2'를 언제부터 시행했는지에 대한 정보이다.  
1년은 보통 52주로 계산한다.  52주보다 큰 값은 없는 것을 확인했으면 이를 datetime으로 변환할 필요가 잆다.  
  
우리에게 주어진 정보는 몇년도 몇주차부터 `Promo2`를 시행했는지에 대한 정보이고, 구체적인 정보(시작일, 기간)는 주어지지 않았다. 일단 여기서 우리는 해당 주차에 '월요일'을 기준으로 `Promo2`를 시행했다고 가정한다. 

In [17]:
from isoweek import Week
Week(2017,12).monday()
Week(2017,12).monday().month, Week(2017,12).monday().day
arr_dt_Promo2 = []
for idx in test_store.index :
    y, w = test_store['Promo2SinceYear'][idx], test_store['Promo2SinceWeek'][idx]
    mon = Week(int(y), int(w)).day(0)
    arr_dt_Promo2.append(dt(int(y), mon.month, mon.day))
arr_dt_Promo2 = pd.Series(arr_dt_Promo2, index=test_store.index)
arr_dt_Promo2.head()
#test_store['dt_Promo2'] = test_store[['Promo2SinceYear','Promo2SinceWeek']].apply(lambda y,w: Week(y,w).monday(), axis=1)
#test_store.head()

1    2010-03-29
2    2011-04-04
10   2012-01-02
11   2010-03-29
12   2009-11-02
dtype: datetime64[ns]

In [18]:
test_store['dt_Promo2'] = arr_dt_Promo2
print(test_store['dt_Promo2'].min(), "~", test_store['dt_Promo2'].max())
test_store.head()

2009-07-27 00:00:00 ~ 2015-06-01 00:00:00


,Store,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,dt_Promo2
1,2,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-03-29
2,3,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2011-04-04
10,11,1,1.0,2012.0,"Jan,Apr,Jul,Oct",2012-01-02
11,12,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-03-29
12,13,1,45.0,2009.0,"Feb,May,Aug,Nov",2009-11-02


`Promo2`를 언제부터 시행했는지에 대한 정보는 사용이 가능하게 변환을 했지만, 결정적으로 한번 프로모션을 하면 얼마나 하는지 알 수가 없다.  
프로모션을 시행하는 간격은 `PromoInterval` 변수에서 정보를 얻을 수 있다.

In [19]:
test_store['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     335
Feb,May,Aug,Nov     130
Mar,Jun,Sept,Dec    106
Name: PromoInterval, dtype: int64

정확히 3개월에 한번씩 자체 프로모션(`Promo2`)를 시행하는 것을 알 수 있다.  
논리적인 비약이 있지만, ~~프로모션을 시행하는 달(month)의 첫날을 기준으로 하여, record에서의 timestamp와의 날짜간격을 변수로 생성할 수 있지 않을까?~~ 라는 생각을 했다.  
일단 문자열로 되어 있는 달(month)정보를 python dictionary 구조를 이용해 key-value pair를 준비해둔다.

In [20]:
month_dic = {'Jan':1,'Apr':4,'Jul':7,'Oct':10, 
             'Feb':2,'May':5,'Aug':8,'Nov':11,
             'Mar':3,'Jun':6,'Sept':9,'Dec':12}
test_record.head()

,Store,dt_date,Sales
0,1,2015-07-31,5263
1,1,2015-07-30,5020
2,1,2015-07-29,4782
3,1,2015-07-28,5011
4,1,2015-07-27,6102


In [21]:
test_merge = pd.merge(test_record, test_store, how='left', on='Store')
test_merge = test_merge.drop(['Promo2SinceWeek', 'Promo2SinceYear'], axis=1)
test_merge.head()
#test_merge[test_merge['Promo2']==1].head()

,Store,dt_date,Sales,Promo2,PromoInterval,dt_Promo2
0,1,2015-07-31,5263,NaN,NaN,NaT
1,1,2015-07-30,5020,NaN,NaN,NaT
2,1,2015-07-29,4782,NaN,NaN,NaT
3,1,2015-07-28,5011,NaN,NaN,NaT
4,1,2015-07-27,6102,NaN,NaN,NaT


In [22]:
from datetime import timedelta
#test_merge['dt_date'].max() + timedelta(days=1)
arr_dt_Promo2 = test_merge['dt_Promo2'].fillna(test_merge['dt_date'].max() + timedelta(days=1))
arr_dt_Promo2.head()

0   2015-08-01
1   2015-08-01
2   2015-08-01
3   2015-08-01
4   2015-08-01
Name: dt_Promo2, dtype: datetime64[ns]

In [23]:
test_merge['dt_Promo2'] = arr_dt_Promo2
test_merge[test_merge['dt_date']==test_merge['dt_date'].max()].head()

,Store,dt_date,Sales,Promo2,PromoInterval,dt_Promo2
0,1,2015-07-31,5263,NaN,NaN,2015-08-01
781,2,2015-07-31,6064,1.0,"Jan,Apr,Jul,Oct",2010-03-29
1565,3,2015-07-31,8314,1.0,"Jan,Apr,Jul,Oct",2011-04-04
2344,4,2015-07-31,13995,NaN,NaN,2015-08-01
3128,5,2015-07-31,4822,NaN,NaN,2015-08-01


`SinceCompetitionOpen` 변수를 처리했을 때와 동일하게, null 값을 갖는 경우는 아예 멀리 보내버려 계산되지 않게 처리했다.  
여기서 최대값보다 하루 뒤로 보낸 이유는 :
> record가 기록된 datetime과 `dt_Promo2` 변수를 비교해서 자체 프로모션을 이미 시작했을 때와 아닐 때를 구분하기 위함이다. 즉, null 값은 아직 자체 프로모션을 시작하지 않은 것과 의미적으로 동일하다는 것이다.  

이제, 이미 자체 프로모션을 시작했다면 `PromoInterval` 에 해당하는 달에 속하는 `Date`는 1 아니면 0을 부여해 binary 변수를 생성하고자 한다.  
일단 자체프로모션을 시작했냐 안했냐 여부를 `tf_Promo2` 변수로 표현할 수 있다.

In [24]:
arr_bin = (test_merge['dt_date'] > test_merge['dt_Promo2'])*1
test_merge['tf_Promo2'] = arr_bin
test_merge[test_merge['dt_date']==test_merge['dt_date'].min()].head()

,Store,dt_date,Sales,Promo2,PromoInterval,dt_Promo2,tf_Promo2
64392,85,2013-01-01,4220,NaN,NaN,2015-08-01,0
195070,259,2013-01-01,6851,NaN,NaN,2015-08-01,0
197575,262,2013-01-01,17267,NaN,NaN,2015-08-01,0
206921,274,2013-01-01,3102,1.0,"Jan,Apr,Jul,Oct",2013-03-04,0
252529,335,2013-01-01,2401,1.0,"Jan,Apr,Jul,Oct",2013-07-29,0


In [25]:
Promo2Months = [month_dic[m] for m in test_merge['PromoInterval'][252529].split(',')]
Promo2Months
#test_merge['dt_date'][252529].month
arr_bin = []
for idx in test_merge.index :
    if test_merge['tf_Promo2'][idx] == 0 :
        arr_bin.append(0)
        continue
    Promo2Months = [month_dic[m] for m in test_merge['PromoInterval'][idx].split(',')]
    if test_merge['dt_date'][idx].month in Promo2Months :
        arr_bin.append(1)
    else :
        arr_bin.append(0)

In [26]:
test_merge['bin_Promo2'] = arr_bin
test_merge[test_merge['dt_date']==dt(2014,12,12)].head()

,Store,dt_date,Sales,Promo2,PromoInterval,dt_Promo2,tf_Promo2,bin_Promo2
189,1,2014-12-12,5269,NaN,NaN,2015-08-01,0,0
971,2,2014-12-12,4407,1.0,"Jan,Apr,Jul,Oct",2010-03-29,1,0
1754,3,2014-12-12,5481,1.0,"Jan,Apr,Jul,Oct",2011-04-04,1,0
2534,4,2014-12-12,8403,NaN,NaN,2015-08-01,0,0
3318,5,2014-12-12,5355,NaN,NaN,2015-08-01,0,0


위 결과는 과거에 자체 프로모션을 이미 시행했으나, 기록된 레코드의 일자가 프로모션 기간에 속하지 않은 예다.  
결과적으로 `tf_Promo2` 변수와 `bin_Promo2` 변수를 곱해
레코드 일자가 자체적인 프로모션을 시행하고 있는 기간인지 아닌지에 대한 여부를 표현하고자 한다.
> 예를들어, 위 결과와 같이 이미 자체프로모션을 시행하고 있으나 행사기간이 아닌 경우에는 0과 1이 곱해져 0으로 표현된다.  
하지만, 반대로 자체프로모션을 시행(`tf_Promo2==1`)하면서 동시에, 레코드 일자가 행사기간에 속할 때(`bin_Promo2==1`)는 1과 1이 곱해져 1로 표현된다.
몰론 자체프로모션을 과거에 시행하지 않았다면(`tf_Promo2==0`) 이므로, 결과는 당연히 0 이다.

binary 변수를 생성하고, 이를 합성해 Promo2에 관련된 변수를 표현할 수 있다.

In [27]:
arr_bin = test_merge['tf_Promo2'] * test_merge['bin_Promo2']
test_merge['res_Promo2'] = arr_bin
test_merge[test_merge['dt_date']==dt(2014,12,12)].head()

,Store,dt_date,Sales,Promo2,PromoInterval,dt_Promo2,tf_Promo2,bin_Promo2,res_Promo2
189,1,2014-12-12,5269,NaN,NaN,2015-08-01,0,0,0
971,2,2014-12-12,4407,1.0,"Jan,Apr,Jul,Oct",2010-03-29,1,0,0
1754,3,2014-12-12,5481,1.0,"Jan,Apr,Jul,Oct",2011-04-04,1,0,0
2534,4,2014-12-12,8403,NaN,NaN,2015-08-01,0,0,0
3318,5,2014-12-12,5355,NaN,NaN,2015-08-01,0,0,0


In [28]:
print("len of merge_table:\t", len(merge_table), "\nlen of test_merge:\t", len(test_merge))
merge_table['IsPromo2'] = test_merge['res_Promo2']
merge_table.head()

len of merge_table:	 844392 
len of test_merge:	 844392


,Store,Date,Sales,Promo,SchoolHoliday,DayOfWeek_4,DayOfWeek_1,DayOfWeek_6,DayOfWeek_7,DayOfWeek_2,...,StoreType_a,StoreType_b,StoreType_d,Assortment_b,Assortment_c,Assortment_a,dt_date,CompetitionDistance,SinceCompetitionOpen,IsPromo2
0,1,2015-07-31,5263,1,1,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-31,1270.0,2524.0,0
1,1,2015-07-30,5020,1,1,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-30,1270.0,2523.0,0
2,1,2015-07-29,4782,1,1,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-29,1270.0,2522.0,0
3,1,2015-07-28,5011,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-28,1270.0,2521.0,0
4,1,2015-07-27,6102,1,1,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,2015-07-27,1270.0,2520.0,0


### modeling of problem 1
위와 같은 과정을 통해 모델에 `Promo2` 변수와 관련된 변수를 추가하였다.

In [29]:
X = merge_table.drop(["Store", "Date", "Sales"], axis=1)
X_train, X_test = X[X['dt_date']<dt(2015,1,1)], X[X['dt_date']>=dt(2015,1,1)]
X_train, X_test = X_train.drop(["dt_date"], axis=1), X_test.drop(["dt_date"], axis=1)

y = merge_table[["Sales", "dt_date"]]
y_train, y_test = y[y['dt_date']<dt(2015,1,1)], y[y['dt_date']>=dt(2015,1,1)]
y_train, y_test = y_train.drop(["dt_date"], axis=1), y_test.drop(["dt_date"], axis=1)

mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)
residual = y_test - y_pred

R2 = metrics.r2_score(y_test, y_pred)
print('R2 Score: \t', R2)

R2 Score: 	 0.215560970496


* * *
## 2. Use Store  
How to differentiate different store ?

1차 과제를 할떄 데이터를 탐색하는 과정에서 매장별 매출분포를 boxplot으로 그려보았다.  
  
![](./Figure_A3_1.png)  

여기서 아이디어를 얻어 매장별 평균 매출 또는 평균 방문 고객수를 반영해 매장정보를 사용하는 것이 어떨까 하는 생각을 해보았다.

처음 merge_table을 생성할 때 합성한 df_train에서 `Customer`에 대한 변수를 제거하였기 때문에 먼저 이를 복원한다.

In [30]:
merge_table.keys()
print("len of merge_table:\t", len(merge_table), "\nlen of df_train:\t", len(raw_train[raw_train['Open']==1]))
df_train = raw_train[raw_train['Open']==1]
df_train.keys()

len of merge_table:	 844392 
len of df_train:	 844392


Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [31]:
test_train = df_train.drop(['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday'], axis=1)
test_train['dt_date'] = pd.to_datetime(test_train['Date'])
test_train.head()

,Store,Date,Sales,Customers,dt_date
0,1,2015-07-31,5263,555,2015-07-31
1,2,2015-07-31,6064,625,2015-07-31
2,3,2015-07-31,8314,821,2015-07-31
3,4,2015-07-31,13995,1498,2015-07-31
4,5,2015-07-31,4822,559,2015-07-31


주(week)차 단위로 평균값을 구하기 위해 각각의 날짜정보가 몇주차에 해당하는지에 대해 조작할 필요가 있다.  
> `datetime` 클래스에 `isocalendar()` 라는 인스턴스 메소드는 datetime 자료형 데이터를 3개의 튜플로 반환한다.  
Return a 3-tuple, (ISO year, ISO week number, ISO weekday).  

이때 우리는 몇주차에 해당하는지가 필요하므로 1번째 인덱스값만 추출해 임시 변수를 생성할 수 있을 것이다.

In [32]:
#type(dt(2017,3,22).isocalendar()[1])
arr_week_num = test_train['dt_date'].apply(lambda x: x.isocalendar()[1])
test_train['tmp_week_num'] = arr_week_num
test_train.head()

,Store,Date,Sales,Customers,dt_date,tmp_week_num
0,1,2015-07-31,5263,555,2015-07-31,31
1,2,2015-07-31,6064,625,2015-07-31,31
2,3,2015-07-31,8314,821,2015-07-31,31
3,4,2015-07-31,13995,1498,2015-07-31,31
4,5,2015-07-31,4822,559,2015-07-31,31


In [33]:
store_avg = test_train.groupby(['Store','tmp_week_num']).mean()
print(type(store_avg))
store_avg.columns = ['W_avg_Sales', 'W_avg_Customers']
store_avg.head()

<class 'pandas.core.frame.DataFrame'>


W_avg_Sales  W_avg_Customers
Store tmp_week_num                              
1     1             4949.500000       604.000000
      2             5048.555556       603.166667
      3             4358.555556       547.777778
      4             4766.166667       570.666667
      5             4689.944444       571.277778

위와 같이 계산한 값은 각 매장의 특성을 나타낸다고 할 수 있다.  
전체기간에서의 평균보다는 각 주차에 따라 평균값을 이용하면, 시계열 정보도 활용할 수 있을 것이라 기대된다.

* * *
## 3. Any other ideas

Ridge 와 **Lasso Regression**을 이용하면 회귀식에서 성능을 높일 수 있지 않을까하는 생각을 했다.  
input 데이터에서 수치형 변수는 별로 없고, 카테고리변수를 더미변수화한 변수가 많기 때문에 일부 불필요한 변수를 0으로 보내는 lasso regression이 더 적절할 것이라 생각된다.